In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import QasmSimulator
from collections import defaultdict
import networkx as nx
from qiskit.circuit import ParameterVector
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler


service = QiskitRuntimeService()

In [25]:
backends = {}
backends["ibm_fez"] = service.backend("ibm_fez", use_fractional_gates=True)
backends["ibm_marrakesh"] = service.backend("ibm_marrakesh")
backends["ibm_torino"] = service.backend("ibm_torino")
backends["ibm_brisbane"] = service.backend("ibm_brisbane")
backends["ibm_sherbrooke"] = service.backend("ibm_sherbrooke")
backends["ibm_kyiv"] = service.backend("ibm_kyiv")
backends["mps"] = service.backend("simulator_mps")
backends["qasm_simulator"] = QasmSimulator()



# 1D-Chain list of qubits

List of qubits in the main 1D-Chain of IBM Eagle and Heron devices. In the main experiment, 100 qubits are used.

In [26]:
qubits_in_line = {}
# Eagle device
qubits_1D_Eagle = list(range(13,-1,-1)) + [14] + list(range(18,33)) + [36] + list(range(51,36,-1)) + [52] + list(range(56,71)) + [74] + list(range(89,74,-1)) + [90] + list(range(94,109)) + [112] + list(range(126,112,-1))

for backend in backends.keys():
    if backend[:3] == "ibm" and backend != "ibm_torino":
        qubits_in_line[backend] = qubits_1D_Eagle

# Heron device
qubits_in_line["ibm_torino"] = list(range(13,-1,-1)) + [15] + list(range(19,34)) + [37] + list(range(52,37,-1)) + [53] + list(range(57,72)) + [75] + list(range(90,75,-1)) + [91] + list(range(95,110)) + [113] + list(range(128,114,-1))
qubits_in_line["ibm_fez"] = list(range(0,16)) + [19] + list(range(35,20,-1)) + [36] + list(range(41,56)) + [59] + list(range(75,60,-1)) + [76] + list(range(81,96)) + [99] + list(range(115,100,-1)) + [116] + list(range(121,124))
qubits_in_line["ibm_marrakesh"] = qubits_in_line["ibm_fez"]

# Ankaa 2
qubits_in_line["ankaa_2"] = []
for i in range(7):
    if i%2:
        qubits_in_line["ankaa_2"]  += [i + j*7 for j in range(12)]
    else:
        qubits_in_line["ankaa_2"]  += [i + j*7 for j in reversed(range(12))]


In [27]:
problems = np.load("./Data/problems_1DChain.npy", allow_pickle=True).item()

# QAOA Circuit 

In [28]:
def maxcut_qaoa_ibm(GT, layers, max_nq, qubits_in_line, fractional_gates=True):
    """
    Constructs a QAOA (Quantum Approximate Optimization Algorithm) circuit for solving the Max-Cut problem 
    on a given weighted graph using IBM quantum hardware constraints.

    Parameters:
    GT (networkx.Graph): The input weighted graph where edge weights represent the cost of a cut.
    layers (int): Number of QAOA layers (depth of the circuit).
    max_nq (int): Maximum number of qubits available for computation.
    qubits_in_line (list): A list specifying the qubit topology (linear connectivity).
    fractional_gates (bool, optional): If True, uses RZZ gates for optimization. 
                                       If False, decomposes RZZ into CX and RZ gates. 
                                       Default is True (preferred for IBM quantum processors).

    Returns:
    QuantumCircuit: A quantum circuit implementing the QAOA Max-Cut algorithm.
    """

    # Define parameter vectors for optimization angles
    betas = ParameterVector("betas", layers)  # Mixing layer rotation angles
    gammas = ParameterVector("gamma", layers)  # Phase-separating layer angles

    # Define two alternating layers for edge connections in a linear qubit topology
    layer_1 = [[qubits_in_line[i], qubits_in_line[i+1]] for i in range(0, len(qubits_in_line)-1, 2)]
    layer_2 = [[qubits_in_line[i], qubits_in_line[i+1]] for i in range(1, len(qubits_in_line)-1, 2)]
    
    # Get the number of nodes in the graph
    nodes = GT.number_of_nodes()
    
    # Find the maximum absolute edge weight in the graph for normalization
    max_weight = np.max(np.abs([GT[i][j]["weight"] for i, j in GT.edges()]))
    
    # Initialize the quantum circuit with max_nq qubits and classical bits for measurement
    qc = QuantumCircuit(max_nq, nodes)
    
    # Get a list of graph nodes
    nodes_list = list(GT.nodes)
    
    # Apply Hadamard gates to all qubits to create an equal superposition state
    qc.h(nodes_list)

    # Apply QAOA layers
    for p in range(layers):
        # Apply phase-separating unitaries using the problem Hamiltonian
        for i, j in GT.edges():
            w = GT[i][j]["weight"]  # Get edge weight
            qbits = [i, j]
            
            # Apply controlled phase shift on layer 1 edges
            if (sorted(qbits) in layer_1 or qbits in layer_1) and abs(w) > 1e-6:
                if fractional_gates:
                    # Use native RZZ gate for more efficient execution on IBM hardware
                    qc.rzz(2 * gammas[p] * GT[i][j]["weight"] / max_weight, *qbits)
                else:
                    # Decomposed version using CX and RZ gates
                    qc.cx(*qbits)
                    qc.rz(2 * gammas[p] * GT[i][j]["weight"] / max_weight, j)
                    qc.cx(*qbits)

        # Apply controlled phase shift on layer 2 edges
        for i, j in GT.edges():
            w = GT[i][j]["weight"]
            qbits = [i, j]
            
            if (sorted(qbits) in layer_2 or qbits in layer_2) and abs(w) > 1e-6:
                if fractional_gates:
                    # Use native RZZ gate if allowed
                    qc.rzz(2 * gammas[p] * GT[i][j]["weight"] / max_weight, *qbits)
                else:
                    # Decomposed implementation
                    qc.cx(*qbits)
                    qc.rz(2 * gammas[p] * GT[i][j]["weight"] / max_weight, j)
                    qc.cx(*qbits)

        # Apply mixing layer with RX rotations to all qubits
        qc.rx(-2 * betas[p], nodes_list)

    # Measure all qubits and store results in classical bits
    qc.measure(nodes_list, range(nodes))

    return qc  # Return the constructed QAOA circuit

def repeat_subgraph(G, qubits_line):
    """
    Repeats a given subgraph structure multiple times across a list of qubits.

    Parameters:
    G (networkx.Graph): The base subgraph to be repeated.
    qubits_line (list): A list of qubit indices where the repeated graph structure will be mapped.

    Returns:
    networkx.Graph: A new graph GT with the repeated subgraph structure.
    """

    n_qubits = G.number_of_nodes()  # Number of nodes in the base subgraph
    n_reps = len(qubits_line) // n_qubits  # Determine how many times the subgraph is repeated

    GT = nx.Graph()  # Create an empty graph
    GT.add_nodes_from(qubits_line[:n_reps * n_qubits])  # Add nodes to GT based on qubits_line

    # Iterate over repetitions and copy edges with weights
    for rep in range(n_reps):
        for i, j in G.edges:
            w = G[i][j]["weight"]  # Extract edge weight
            # Map original nodes to repeated positions and add weighted edges
            GT.add_weighted_edges_from([[qubits_line[i + rep * n_qubits], qubits_line[j + rep * n_qubits], w]])

    return GT  # Return the expanded graph with repeated subgraphs

# Objective Function

In [29]:
def cost_maxcut(bitstring, weights):
    """
    Computes the cost of a given bitstring solution for the Max-Cut problem.

    Parameters:
    bitstring (str): A binary string representing a partition of the graph nodes (e.g., "1010").
    weights (dict): A dictionary where keys are edge tuples (i, j) and values are edge weights.

    Returns:
    float: The computed cost of the Max-Cut solution.
    """
    cost = 0  # Initialize the cost
    
    # Iterate through all edges in the graph
    for i, j in weights.keys():
        # Check if the nodes i and j are in different partitions (cut condition)
        if bitstring[i] + bitstring[j] in ["10", "01"]:
            cost += weights[i, j]  # Add the edge weight to the cost

    return cost  # Return the total cut cost


def objective_MaxCut(samples_dict, G, optimal):
    """
    Evaluates the performance of a quantum algorithm for the Max-Cut problem.

    Parameters:
    samples_dict (dict): A dictionary where keys are bitstrings (binary solutions), 
                         and values are their occurrence counts.
    G (networkx.Graph): The input weighted graph where edges represent cut costs.
    optimal (str): The optimal bitstring solution found by classical solvers (e.g., CPLEX).

    Returns:
    dict: A dictionary containing:
        - "results": A numpy array with computed cost, normalized cost ratio, and counts.
        - "G": The input graph G.
        - "weights": The edge weights extracted from G.
        - "max_cut": The cost of the optimal Max-Cut solution.
        - "r": The expected approximation ratio.
        - "probability": The probability of sampling the optimal solution.
    """

    # Extract weights from the graph's edges
    weights = {(i, j): (G[i][j]["weight"] if len(G[i][j]) != 0 else 1) for i, j in G.edges}
    
    # Compute the cost of the optimal Max-Cut solution
    max_cost = cost_maxcut(optimal, weights)

    results = []  # Stores results in the form [cost, ratio, counts]
    probability = 0  # Tracks probability of sampling the optimal solution

    # Iterate through all sampled bitstrings
    for bitstring, counts in samples_dict.items():
        cost = cost_maxcut(bitstring, weights)  # Compute cost of the given bitstring
        r = cost / max_cost  # Compute the cost ratio relative to the optimal solution
        results.append([cost, r, counts])  # Store results
        
        # If this bitstring matches the optimal cost, update probability
        if abs(cost - max_cost) < 1e-6:
            probability += counts
        
        # Check if a better-than-optimal solution appears (sanity check)
        if cost > max_cost:
            print(f"There is a better cost than that of CPLEX: {cost - max_cost}")

    # Convert results to a NumPy array for easy computation
    results = np.array(results)

    # Total number of shots (total sampled solutions)
    shots = np.sum(results[:, 2])

    # Compute the expected approximation ratio: (weighted sum of costs) / (shots * max_cost)
    rT = np.sum(results[:, 0] * results[:, 2]) / (shots * max_cost)

    # Normalize the probability of sampling the optimal solution
    probability /= shots

    # Return results in a structured dictionary
    return {
        "results": np.array(results),
        "G": G,
        "weights": weights,
        "max_cut": max_cost,
        "r": rT,
        "probability": probability
    }

def mitigate(samples_dict, G, random=False):
    """
    Applies error mitigation by flipping individual bits in sampled solutions 
    to find better Max-Cut solutions.

    Parameters:
    samples_dict (dict): A dictionary where keys are bitstrings (binary solutions), 
                         and values are their occurrence counts.
    G (networkx.Graph): The input weighted graph where edges represent cut costs.
    random (bool, optional): If True, randomizes the order in which qubits are flipped.
                             Default is False (systematic flipping).

    Returns:
    dict: A dictionary of improved bitstring samples with their updated counts.
    """

    # Define a mapping to flip bits ('0' -> '1', '1' -> '0')
    change = {"0": "1", "1": "0"}

    # Get the number of nodes (qubits)
    nq = G.number_of_nodes()

    # Extract weights from the graph's edges
    weights = {(i, j): (G[i][j]["weight"] if len(G[i][j]) != 0 else 1) for i, j in G.edges}

    # Dictionary to store new (improved) samples
    new_samples = defaultdict(int)

    # Iterate over all bitstring samples
    for bitstring, counts in samples_dict.items():
        for _ in range(counts):  # Process each occurrence of the bitstring separately
            best_string = bitstring  # Initialize the best solution as the current one
            best_cost = cost_maxcut(bitstring, weights)  # Compute its cost
            
            # Create an ordered list of qubits (nodes) to consider flipping
            list_qubits = np.arange(nq)
            
            # If random flipping is enabled, shuffle the qubit order
            if random:
                np.random.shuffle(list_qubits)

            # Try flipping each qubit and check if the cost improves
            for qi in list_qubits:
                # Flip the bit at position qi
                new_string = "".join((change[i] if n == qi else i) for n, i in enumerate(best_string))
                new_cost = cost_maxcut(new_string, weights)

                # If the new configuration gives a better cost, update the best solution
                if new_cost > best_cost:
                    best_string = new_string
                    best_cost = new_cost
            
            # Store the improved bitstring in the new_samples dictionary
            new_samples[best_string] += 1

    return new_samples  # Return the mitigated samples

def random_samples(num_samples, n_qubits):
    """
    Generates random bitstring samples for a given number of qubits.

    Parameters:
    num_samples (int): The number of random bitstrings to generate.
    n_qubits (int): The number of qubits (length of each bitstring).

    Returns:
    dict: A dictionary where keys are randomly generated bitstrings 
          and values are their occurrence counts.
    """
    
    random_samples = defaultdict(int)  # Dictionary to store bitstrings and their counts

    # Generate random bitstrings and count their occurrences
    for _ in range(num_samples):
        bitstring = "".join(str(i) for i in np.random.choice([0, 1], n_qubits))  # Generate a random bitstring
        random_samples[bitstring] += 1  # Increment count for the generated bitstring

    return random_samples  # Return the dictionary of samples


# 1) IBM Eagle and Heron: Prepare the experiments

In [31]:
# Select the backend for quantum computation
# Uncomment the desired backend or set the active one
# backend_name = "ibm_brisbane"
# backend_name = "ibm_sherbrooke"
# backend_name = "ibm_kyiv"
# backend_name = "ibm_nazca"
# backend_name = "ibm_osaka"
# backend_name = "ibm_kyoto"
# backend_name = "ibm_torino"
backend_name = "ibm_fez"  # Active backend
# backend_name = "ibm_marrakesh"
# backend_name = "ibm_strasbourg"
# backend_name = "ibm_brussels"
# backend_name = "ankaa_2"
# backend_name = "qasm_simulator"  # Classical simulator option

results = {}  # Dictionary to store experiment results

# ---------------------- Loading Problem Information ----------------------

nq = 100  # Number of qubits (size of the problem)

# Define qubit mappings for different backends
qubits_in_line["qasm_simulator"] = range(nq)  # Simulator uses a linear qubit range

# Load the problem graph
results["G"] = problems["G"][nq]  

# Repeat subgraph structure to match the qubit layout of the chosen backend
results["GT"] = repeat_subgraph(results["G"], qubits_in_line[backend_name])

# Select the graph to use: 
# If using a real quantum backend, use the transformed graph (GT), 
# otherwise use the original problem graph (G) for the simulator
GT = results["GT"] if backend_name != "qasm_simulator" else results["G"]

# Load optimal solutions for the given problem size
sols = problems["sol"][nq]  

# Get the list of qubits assigned for the selected backend
qubits_line = qubits_in_line[backend_name]

# Get the maximum number of qubits available on the backend
max_nq = backends[backend_name].num_qubits  

# ---------------------- QAOA Configuration ----------------------

# Define the number of QAOA layers
ps = [3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 20, 25, 30, 40, 50, 75, 100]  
# ps = [10_000]  # Set a single large QAOA depth for testing

# Define delta values for parameter scaling
# deltas = np.linspace(0.5, 1.5, 10)  # Generate 10 values between 0.5 and 1.5
deltas = [0.75]  # Set a single delta value for testing

# Store key experiment parameters in results
results["optimal"] = sols  
results["nq"] = nq  
results["ps"] = ps  
results["Deltas"] = deltas  

# ---------------------- Circuit Compilation ----------------------

circuits_transpiled = {}  # Store transpiled circuits for different QAOA layers

# Iterate over different QAOA layers
for p in ps:
    print(f"Layer: -------  {p} ----------- ")  # Display current layer
    
    # Generate the QAOA circuit for MaxCut problem
    circ_maxcut = maxcut_qaoa_ibm(GT, p, max_nq, qubits_line, fractional_gates=True)
    
    # Transpile the circuit for the selected backend
    # Optimization level 1 applies basic circuit optimizations while preserving logical structure
    backend_circ = transpile(circ_maxcut, backend=backends[backend_name], optimization_level=1, initial_layout=range(max_nq))
    
    # Store the transpiled circuit
    circuits_transpiled[p] = backend_circ  

# ---------------------- Assigning Parameters & Executing Circuits ----------------------

circuits = []  # Store final circuits with assigned parameters

# Iterate over different delta values
for delta in deltas:
    print(f"Delta: -------  {round(delta,2)} ----------- ")  # Display current delta value
    
    for p in ps:
        # Compute beta and gamma parameters for QAOA
        betas = list(np.arange(1, p+1)[::-1] * delta/p)  # Reverse sequence for betas
        gammas = list(np.arange(1, p+1) * delta/p)  # Forward sequence for gammas
        
        # Assign the computed parameters to the transpiled circuit
        backend_circ = circuits_transpiled[p].assign_parameters(np.concatenate((betas, gammas)))
        
        # Store the final circuit
        circuits.append(backend_circ)  


Layer: -------  3 ----------- 
Layer: -------  4 ----------- 
Layer: -------  5 ----------- 
Layer: -------  6 ----------- 
Layer: -------  7 ----------- 
Layer: -------  8 ----------- 
Layer: -------  9 ----------- 
Layer: -------  10 ----------- 
Layer: -------  13 ----------- 
Layer: -------  15 ----------- 
Layer: -------  20 ----------- 
Layer: -------  25 ----------- 
Layer: -------  30 ----------- 
Layer: -------  40 ----------- 
Layer: -------  50 ----------- 
Layer: -------  75 ----------- 
Layer: -------  100 ----------- 
Delta: -------  0.75 ----------- 


# 2) IBM Eagle and Heron: Run the experiments

In [33]:
shots = 1_000  # Number of measurement shots per circuit execution
extra = ""  # Additional identifier for file naming (indicating 10,000 QAOA layers)
# extra = ""  # Uncomment this line if no extra identifier is needed
results["shots"] = shots  

# Check if using a real quantum backend (not the classical simulator)
if backend_name != "qasm_simulator":
    # Initialize a sampler for running the circuits on a real quantum device
    sampler = Sampler(mode=backends[backend_name])
    
    # Submit the circuits for execution with the specified number of shots
    submit_job = sampler.run(circuits, shots=shots)
    
    # Store the job ID to track or retrieve results later
    results["id"] = submit_job.job_id()  
    
    # Alternative way to submit the job directly to the backend
    # submit_job = backends[backend_name].run(circuits, shots=shots)
    # results["id"] = submit_job.job_id()  # Save job ID for retrieval

else:
    # If using the qasm_simulator (classical simulator), execute the circuits directly
    dict_results = backends[backend_name].run(circuits, shots=shots).result().get_counts()
    
    # Process the results:
    # - Reverse bitstrings (to match standard qubit ordering)
    # - Organize results into a nested dictionary: {delta -> {p -> {bitstring -> counts}}}
    results["samples"] = {
        delta: {
            p: {k[::-1]: v for k, v in dict_results[i + nd * len(ps)].items()} 
            for i, p in enumerate(results["ps"])
        } 
        for nd, delta in enumerate(results["Deltas"])
    }

# Save results as a NumPy binary file for further analysis
np.save(f"./Data/{backend_name}/{nq}_1D{extra}.npy", results)  


# Retrieve the experimental information

In [10]:
backend_name = "ibm_fez"  # Define the IBM backend used for execution

# Specify additional identifier for file naming:
extra = ""  # No extra identifier

nq = 100  # Number of qubits

# Load the previously saved results from a NumPy binary file
results = np.load(f"./Data/{backend_name}/{nq}_1D{extra}.npy", allow_pickle=True).item()

# If using a real quantum device, retrieve job results from IBM Quantum service
if backend_name != "qasm_simulator":
    # Fetch the job results using its stored job ID
    job = service.job(job_id=results["id"]).result()

    # Extract bitstring measurement results from all circuits in the job
    dict_results = [job[i].data.c.get_counts() for i in range(len(job))]

    # Process and store results:
    # - Reverse bitstrings (`k[::-1]`) to match standard qubit ordering.
    # - Organize results into a nested dictionary: {delta -> {p -> {bitstring -> counts}}}.
    results["samples"] = {
        delta: {
            p: {k[::-1]: v for k, v in dict_results[i + nd * len(ps)].items()}
            for i, p in enumerate(results["ps"])
        }
        for nd, delta in enumerate(results["Deltas"])
    }


# Postprocessing the samples

In [11]:
# Extract the number of nodes (qubits) in the original and repeated graph
nq = results["G"].number_of_nodes()  # Number of qubits in the original problem graph
nq_total = results["GT"].number_of_nodes()  # Number of qubits in the repeated subgraph
sections = nq_total // nq  # Number of repeated sections in the larger subgraph

# Dictionaries to store post-processing results
postprocessing = {}
postprocessing_mitig = {}

# Iterate over different values of delta (hyperparameter for QAOA)
for delta in results["samples"]:
    postprocessing[delta] = {}
    postprocessing_mitig[delta] = {}

    # Iterate over different QAOA layer depths (p)
    for p in results["samples"][delta]:
        print(f"----------- p = {p} -------------")
        postprocessing[delta][p] = {}
        postprocessing_mitig[delta][p] = {}

        # Iterate over each section of the repeated subgraph
        for sec in range(sections):
            samples_sec = defaultdict(int)

            # Extract samples corresponding to the current section
            for k, v in results["samples"][delta][p].items():
                samples_sec[k[sec * nq : (sec + 1) * nq]] += v

            # Compute objective function for the MaxCut problem
            postprocessing[delta][p][sec] = objective_MaxCut(samples_sec, results["G"], results["optimal"][0])

            # Apply error mitigation (local search improvement)
            new_samples = mitigate(samples_sec, results["G"], random=False)

            # Compute the objective function for mitigated results
            postprocessing_mitig[delta][p][sec] = objective_MaxCut(new_samples, results["G"], results["optimal"][0])

# Store the post-processed and mitigated results
results["postprocessing"] = postprocessing
results["postprocessing_mitig"] = postprocessing_mitig

# Generate random bitstring samples for comparison
rand_samples = random_samples(10_000, nq)

# Compute the objective function for random sampling (baseline performance)
results["random"] = objective_MaxCut(rand_samples, results["G"], results["optimal"][0])

# Apply error mitigation to random samples and evaluate performance
results["random_mitig"] = objective_MaxCut(mitigate(rand_samples, results["G"], random=False), results["G"], results["optimal"][0])

# Save the updated results dictionary
np.save(f"./Data/{backend_name}/{nq}_1D{extra}.npy", results)


----------- p = 10000 -------------
